---
## 전체 http 다 갖고 오는 것
---

In [1]:
# MODULE IMPORT & ETC
#======================================================================================================================#
import pandas as pd
import os
import gensim
import numpy as np
import time
import pickle
import warnings
from datetime import datetime
from datetime import date, timedelta
from imblearn.over_sampling import RandomOverSampler
from gensim.models.fasttext import FastText as FT_gensim
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Conv1D, GlobalMaxPool1D, SimpleRNN
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from DNS_Encoding.data_nodns_del import data_nodns_del
from DNS_Encoding.make_fasttext_model_vector2 import make_fasttext_model_vector2
from DNS_Encoding.making_training_bydns import making_training_bydns
import pymysql
from IP_Encoding.ipencoding_65536_delH import parallelize_dataframe
from IP_Encoding.ipencoding_65536_delH import ipencoding_65536_delH
from Etc_features.age_onehot import age_onehot
from Etc_features.gender_onehot import gender_onehot
from Etc_features.port_onehot import port_onehot
from Etc_features.protocol_onehot import protocol_onehot
from Etc_features.train_lastdnstimetolive import train_lastdnstimetolive
from Etc_features.train_delta_dn_link_data_size import train_delta_dn_link_data_size
from Etc_features.train_delta_up_link_data_size import train_delta_up_link_data_size
from Etc_features.train_synackmsssize import train_synackmsssize
from Etc_features.train_delta_dn_link_payload_size import train_delta_dn_link_payload_size
from Etc_features.train_delta_up_link_data_size import train_delta_up_link_data_size
from DNS_Encoding.data_nodns_unknown_del import data_nodns_unknown_del
from Etc_features.train_delta_up_link_payload_size import train_delta_up_link_payload_size
from Etc_features.train_delta_dn_link_payload_size import train_delta_dn_link_payload_size



warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# ======================================================================================================================#
# Input, Save data name
# ======================================================================================================================#
ExecutionTime = datetime.now()
CREATE_TIME = pd.to_datetime(ExecutionTime)

### 시스템용. 
# Input data
#current = str(datetime.now())[:13] + ":00"
#before1hour = str(datetime.now() - timedelta(hours=1))
#before1hour = before1hour[:13] + ":00"
#datatable = before1hour.replace("-", "")[:8]
#before1hour_date = 'T_AD_XDR_HTTP_KNOWN_' + datatable


###  데모용
# 테스트용
current = '2020-01-21 01:00'
before1hour = '2020-01-21 00:00'
datatable = '20200121'
before1hour_date = 'T_AD_XDR_HTTP_KNOWN_' + datatable


# 저장할 모델 이름



#======================================================================================================================#
# Data Load
#======================================================================================================================#

#http = pd.read_csv('http_classifier_train_20191212.csv')

conn = pymysql.connect(host ='192.168.6.89' , port = 3306,
                       user = 'atom', password = 'atom', db = '5G_Probe_App_Discovery', charset = 'utf8',
                      cursorclass = pymysql.cursors.DictCursor)

# 커서 가져오기 - 시간 정확히 다시 확인 필요
curs = conn.cursor()


# 전체 데이터 가져오기. 
sql = "SELECT * FROM " + before1hour_date

# 1시간짜리 데이터만 가져오기. 
# sql = "SELECT * FROM " + before1hour_date + " WHERE OngoingFlag = 1 AND SummaryCreateTime >= unix_timestamp(" +"'"+before1hour+"'"+") AND SummaryCreateTime < unix_timestamp(" +"'"+ current +"');"


curs.execute(sql)

# 데이터 가져오기
http = pd.DataFrame(curs.fetchall())
http = http[['SummaryCreateTime', 'IMSI', 'Gender', 'Age', 'App_ID', 'App_Group_Code', 'App_Port','Protocol', 'IP_Version', 'App_IPv4', 'App_IPv6',  'DNS.QueryName', 'HTTP.Request.Host', 'DNS.LastDnsTimeToLive', 'Traffic.Delta.Dnlink.DataSize', 'Traffic.Delta.Uplink.DataSize', 'TCP.Delta.MSS.SynMssSize', 'TCP.Delta.MSS.SynAckMssSize', 'Traffic.Delta.Uplink.RetransPayloadSize', 'Traffic.Delta.Dnlink.RetransPayloadSize', 'HTTP.Delta.Quility.TotResponseTime']].copy()


# ======================================================================================================================#
# Preprocessing
# ======================================================================================================================#
# ----------------------- 1. 동일Rule인데 app_id가 다른 경우 삭제
a = http.reset_index()
group1 = a.groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).agg(
    {"App_ID": lambda x: x.nunique()}).reset_index()
group2 = a.groupby(['Protocol', 'App_Port', 'DNS.QueryName', 'App_IPv4', 'App_IPv6'])['index'].unique().reset_index()

group1 = group1[group1['App_ID'] >= 2].index.tolist()
group2 = group2[group2.index.isin(group1)].reset_index(drop=True)

index_du = []
for i in range(len(group2)):
    index_du += list(group2['index'][i])

index_du.sort()
http = http[~http.index.isin(index_du)].reset_index(drop=True)


Using TensorFlow backend.


In [2]:
http['new_label'] = http['App_ID']

In [4]:
appid = list(http.groupby(['App_ID']).count().index)

## 개쩌는 Broadcasting

In [5]:
len(http[http.App_ID == appid[0]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index)

241

## 이렇게 해야 copy 값이 return 된다.

In [6]:
xx = ((http[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss)).sum(axis=1)==5)
http.loc[xx.index[xx], 'new_label'] = 'ccc'

NameError: name 'ss' is not defined

## Rule이 6개이상인 appid만 뽑아내자 : 548개중 245개다. 

In [7]:
appid_6 = []
for i in range(len(appid)):
    if len(http[http.App_ID == appid[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index) >5:
        appid_6.append(appid[i])

In [ ]:
len(appid),len(appid_6)

## 한 15초 걸림

In [8]:
for i in range(len(appid_6)):
    
    if i < len(appid_6)-1:
    # rule 2개만 뽑는다. 
        ss_1 = http[http.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[0]
        ss_2 = http[http.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[1]
       
        xx_1 = ((http[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss_1)).sum(axis=1)==5)
        xx_2 = ((http[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss_2)).sum(axis=1)==5)
        
    # rule 2개의 new_label을 다음 것으로 바꿔준다.     
        http.loc[xx_1.index[xx_1], 'new_label'] = appid_6[i+1]
        http.loc[xx_2.index[xx_2], 'new_label'] = appid_6[i+1]
        
    # 마지막 것 첫번재로 넘겨주는 것    
    else:
        ss_1 = http[http.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[0]
        ss_2 = http[http.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[1]
       
        xx_1 = ((http[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss_1)).sum(axis=1)==5)
        xx_2 = ((http[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss_2)).sum(axis=1)==5)
        
    # rule 2개의 new_label을 다음 것으로 바꿔준다.     
        http.loc[xx_1.index[xx_1], 'new_label'] = appid_6[0]
        http.loc[xx_2.index[xx_2], 'new_label'] = appid_6[0]

## Isolation Forest

In [10]:
#======================================================================================================================#

# ----------------------- 0. 정확도 판단을 위해 appid rule별 데이터 만들기

# ----------------------- 2. IP encoding

http_ip65536_delH = parallelize_dataframe(http, ipencoding_65536_delH)

# ----------------------- 3. DNS encoding

http_1 = data_nodns_unknown_del(http)
http_vector, http_modelinit, max_length_http = make_fasttext_model_vector2(http_1)
httpdns_training = making_training_bydns(http, http_vector)

# ----------------------- 4.  ETC Features encoding

http_age = age_onehot(http)
http_gender = gender_onehot(http)
http_port = port_onehot(http)
http_protocol = protocol_onehot(http)
http_lastdnstimetolive = train_lastdnstimetolive(http)
http_delta_dn_link_data_size = train_delta_dn_link_data_size(http)
http_delta_up_link_data_size = train_delta_up_link_data_size(http)
http_synackmsssize = train_synackmsssize(http)
http_delta_dn_link_payload_size = train_delta_dn_link_payload_size(http)
http_delta_up_link_payload_size = train_delta_up_link_payload_size(http)

http_training_ip65536_delH = pd.concat(
    [http_ip65536_delH, httpdns_training, http_port, http_protocol,
                                             http_lastdnstimetolive,http_delta_dn_link_data_size,http_delta_up_link_data_size,http_synackmsssize,http_delta_dn_link_payload_size,
                                             http_delta_up_link_payload_size], axis=1)


## appid와 newlabel도 포함시킨 것.

In [11]:
http_training_ip65536_delH = pd.concat([http_training_ip65536_delH, http[['App_ID']], http[['new_label']]], axis = 1)

## IF는 new_label이 6개이상인것만 돌린다. 

In [12]:
a = pd.DataFrame(columns = http_training_ip65536_delH.columns)

for i in range(len(appid)):
    if len(http_training_ip65536_delH[http_training_ip65536_delH.new_label == appid[i]])<6:
        xx = http_training_ip65536_delH.loc[list(http_training_ip65536_delH[http_training_ip65536_delH.new_label == appid[i]].index)]
        xx['anomaly_score'] = [2 for i in range(len(xx))]
        a = pd.concat([a,xx])

            
    else:
        clf = IsolationForest(n_estimators=50, contamination=0.2, random_state=35, verbose=1)
        clf.fit(http_training_ip65536_delH[http_training_ip65536_delH.new_label == appid[i]].drop(['App_ID','new_label'], axis = 1))
        y_pred = clf.score_samples(http_training_ip65536_delH[http_training_ip65536_delH.new_label == appid[i]].drop(['App_ID','new_label'], axis = 1)) #1,-1로 나온다.
        xx = http_training_ip65536_delH.loc[list(http_training_ip65536_delH[http_training_ip65536_delH.new_label == appid[i]].index)]
        xx['anomaly_score'] = list(y_pred)
        a = pd.concat([a,xx])
        
        
###### -1이 이상치, 1이 정상.

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

In [13]:
a

,Aclass,Bclass,Cclass,Dclass,Eclass,Fclass,Gclass,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,port__80,port__8080,port_x,protocol__6,protocol_x,lastdnstimetolive,delta_dn_link_data_size,delta_up_link_data_size,synackmsssize,delta_dn_link_payload_size,delta_up_link_payload_size,App_ID,new_label,anomaly_score
33,0.001526,0.998474,0.0,0.0,0.0,0.0,0.705913,6.264721,-1.489101,1.482919,-1.435188,1.301602,1.233277,0.900699,1.443186,-1.782538,0.869659,4.701369,-0.458549,0.920637,-1.265305,1.122995,4.276284,-0.577299,0.943739,-0.989030,0.850424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0.0,7.025218e-07,2.062145e-06,0.0,0.0,0.0,C350,C350,-0.373416
34,0.001526,0.998474,0.0,0.0,0.0,0.0,0.870848,6.264721,-1.489101,1.482919,-1.435188,1.301602,1.233277,0.900699,1.443186,-1.782538,0.869659,4.701369,-0.458549,0.920637,-1.265305,1.122995,4.276284,-0.577299,0.943739,-0.989030,0.850424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0.0,7.039497e-07,1.986361e-06,0.0,0.0,0.0,C350,C350,-0.378240
35,0.001526,0.998474,0.0,0.0,0.0,0.0,0.107027,0.140844,-1.112594,2.754042,-1.823551,0.187387,1.233277,0.900699,1.443186,-1.782538,0.869659,-0.551427,0.198148,0.888071,-0.400201,0.224195,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0.0,2.912895e-06,4.295804e-06,0.0,0.0,0.0,C350,C350,-0.448223
67,0.001526,0.998474,0.0,0.0,0.0,0.0,0.828031,-0.591596,-1.031687,1.811883,-2.860316,1.168687,-0.432445,-2.248495,3.465404,-2.433781,0.460784,1.233277,0.900699,1.443186,-1.782538,0.869659,-0.551427,0.198148,0.888071,-0.400201,0.224195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0.0,3.229887e-06,2.504889e-05,0.0,0.0,0.0,C350,C350,-0.406918
84,0.001526,0.998474,0.0,0.0,0.0,0.0,0.107027,0.140844,-1.112594,2.754042,-1.823551,0.187387,1.233277,0.900699,1.443186,-1.782538,0.869659,-0.551427,0.198148,0.888071,-0.400201,0.224195,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0.0,5.738546e-04,4.630853e-05,0.0,0.0,0.0,C350,C350,-0.506357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198859,0.001526,0.998474,0.0,0.0,0.0,0.0,0.204318,8.020286,-5.343295,0.801315,-1.901989,3.373604,1.105336,-0.240716,-0.505421,0.190869,1.249949,4.701369,-0.458549,0.920637,-1.265305,1.122995,4.276284,-0.577299,0.943739,-0.989030,0.850424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0.0,2.918607e-06,5.101517e-06,0.0,0.0,0.0,IABP,IABP,-0.406597
198954,0.001526,0.998474,0.0,0.0,0.0,0.0,0.204318,8.020286,-5.343295,0.801315,-1.901989,3.373604,1.105336,-0.240716,-0.505421,0.190869,1.249949,4.701369,-0.458549,0.920637,-1.265305,1.122995,4.276284,-0.577299,0.943739,-0.989030,0.850424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0.0,1.783863e-05,4.126684e-05,0.0,0.0,0.0,IABP,IABP,-0.530681
198998,0.001526,0.998474,0.0,0.0,0.0,0.0,0.204318,0.024478,0.001233,-0.194493,0.109131,0.153056,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0.0,1.227985e-07,6.860523e-07,0.0,0.0,0.0,IABP,IABP,-0.559112
199073,0.001526,0.998474,0.0,0.0,0.0,0.0,0.204318,0.024478,0.001233,-0.194493,0.109131,0.153056,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0.0,2.455970e-07,1.372105e-06,0.0,0.0,0.0,IABP,IABP,-0.575906


In [14]:
http = pd.concat([http, a[['anomaly_score']]], axis = 1)

## 2개씩 뽑은 rule error 개수 및 2개씩 뽑은 것중 error로 알고리즘이 판단한 것 개수

In [44]:
every = 0
anomaly = 0


for i in range(len(appid_6)):
    ss_1 = http[http.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[0]
    ss_2 = http[http.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[1]
       
    xx_1 = ((http[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss_1)).sum(axis=1)==5)
    xx_2 = ((http[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss_2)).sum(axis=1)==5)

        
    every += 2
        
        
    if sum(http.loc[xx_1.index[xx_1], 'anomaly_score'] <-0.45)/len(http.loc[xx_1.index[xx_1], 'anomaly_score']) >= 0.6:
        anomaly += 1
            
    if sum(http.loc[xx_2.index[xx_2], 'anomaly_score'] <-0.55)/len(http.loc[xx_2.index[xx_2], 'anomaly_score']) >= 0.6:
        anomaly += 1
      
    
    

IndexError: index 0 is out of bounds for axis 0 with size 0

## 알고리즘이 판단한 rule error 개수

In [35]:
algorithm = 0

for i in range(len(appid_6)):
    
    for j in range(len(http[http.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index)):
  
        ss = http[http.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[j]
        xx = ((http[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss)).sum(axis=1)==5)
        
        
        if sum(xx) >= 6:
        # 이게 각 rule 별 anomaly_score다.    
            if sum(http.loc[xx.index[xx], 'anomaly_score'] <-0.5)/len(http.loc[xx.index[xx]]) >= 0.7:
                algorithm += 1

## 전체 Rule 개수

In [23]:
everything = 0

for i in range(len(appid)):
    
    for j in range(len(http[http.App_ID == appid[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index)):
  
        everything +=1

In [24]:
everything

8692

In [36]:
algorithm

601

In [34]:
anomaly, every, anomaly/every

(400, 490, 0.8163265306122449)

---
## transport
---

In [37]:
# MODULE IMPORT & ETC
#======================================================================================================================#
import pandas as pd
import os
import gensim
import numpy as np
import time
import pickle
import warnings
from datetime import datetime
from datetime import date, timedelta
from imblearn.over_sampling import RandomOverSampler
from gensim.models.fasttext import FastText as FT_gensim
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Conv1D, GlobalMaxPool1D, SimpleRNN
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from DNS_Encoding.data_nodns_del import data_nodns_del
from DNS_Encoding.make_fasttext_model_vector2 import make_fasttext_model_vector2
from DNS_Encoding.making_training_bydns import making_training_bydns
import pymysql
from IP_Encoding.ipencoding_65536_delH import parallelize_dataframe
from IP_Encoding.ipencoding_65536_delH import ipencoding_65536_delH
from Etc_features.age_onehot import age_onehot
from Etc_features.gender_onehot import gender_onehot
from Etc_features.port_onehot import port_onehot
from Etc_features.protocol_onehot import protocol_onehot
from Etc_features.train_lastdnstimetolive import train_lastdnstimetolive
from Etc_features.train_delta_dn_link_data_size import train_delta_dn_link_data_size
from Etc_features.train_delta_up_link_data_size import train_delta_up_link_data_size
from Etc_features.train_synackmsssize import train_synackmsssize
from Etc_features.train_delta_dn_link_payload_size import train_delta_dn_link_payload_size
from Etc_features.train_delta_up_link_data_size import train_delta_up_link_data_size
from DNS_Encoding.data_nodns_unknown_del import data_nodns_unknown_del
from Etc_features.train_delta_up_link_payload_size import train_delta_up_link_payload_size
from Etc_features.train_delta_dn_link_payload_size import train_delta_dn_link_payload_size



warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# ======================================================================================================================#
# Input, Save data name
# ======================================================================================================================#
ExecutionTime = datetime.now()
CREATE_TIME = pd.to_datetime(ExecutionTime)

### 시스템용. 
# Input data
#current = str(datetime.now())[:13] + ":00"
#before1hour = str(datetime.now() - timedelta(hours=1))
#before1hour = before1hour[:13] + ":00"
#datatable = before1hour.replace("-", "")[:8]
#before1hour_date = 'T_AD_XDR_HTTP_KNOWN_' + datatable


###  데모용
# 테스트용
current = '2020-01-21 01:00'
before1hour = '2020-01-21 00:00'
datatable = '20200121'
before1hour_date = 'T_AD_XDR_TR_KNOWN_' + datatable


# 저장할 모델 이름



#======================================================================================================================#
# Data Load
#======================================================================================================================#

#http = pd.read_csv('http_classifier_train_20191212.csv')

conn = pymysql.connect(host ='192.168.6.89' , port = 3306,
                       user = 'atom', password = 'atom', db = '5G_Probe_App_Discovery', charset = 'utf8',
                      cursorclass = pymysql.cursors.DictCursor)

# 커서 가져오기 - 시간 정확히 다시 확인 필요
curs = conn.cursor()


# 전체 데이터 가져오기. 
sql = "SELECT * FROM " + before1hour_date

# 1시간짜리 데이터만 가져오기. 
# sql = "SELECT * FROM " + before1hour_date + " WHERE OngoingFlag = 1 AND SummaryCreateTime >= unix_timestamp(" +"'"+before1hour+"'"+") AND SummaryCreateTime < unix_timestamp(" +"'"+ current +"');"


curs.execute(sql)

# 데이터 가져오기
transport = pd.DataFrame(curs.fetchall())
transport = transport[['SummaryCreateTime', 'IMSI', 'Gender', 'Age', 'App_ID', 'App_Group_Code', 'App_Port','Protocol', 'IP_Version', 'App_IPv4', 'App_IPv6',  'DNS.QueryName', 'DNS.LastDnsTimeToLive', 'Traffic.Delta.Dnlink.DataSize', 'Traffic.Delta.Uplink.DataSize', 'TCP.Delta.MSS.SynMssSize', 'TCP.Delta.MSS.SynAckMssSize', 'Traffic.Delta.Uplink.RetransPayloadSize', 'Traffic.Delta.Dnlink.RetransPayloadSize']].copy()

# ======================================================================================================================#
# Preprocessing
# ======================================================================================================================#
# ----------------------- 1. 동일Rule인데 app_id가 다른 경우 삭제
a = transport.reset_index()
group1 = a.groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).agg(
    {"App_ID": lambda x: x.nunique()}).reset_index()
group2 = a.groupby(['Protocol', 'App_Port', 'DNS.QueryName', 'App_IPv4', 'App_IPv6'])['index'].unique().reset_index()

group1 = group1[group1['App_ID'] >= 2].index.tolist()
group2 = group2[group2.index.isin(group1)].reset_index(drop=True)

index_du = []
for i in range(len(group2)):
    index_du += list(group2['index'][i])

index_du.sort()
transport = transport[~transport.index.isin(index_du)].reset_index(drop=True)


In [39]:
transport['new_label'] = transport['App_ID']
appid = list(transport.groupby(['App_ID']).count().index)

appid_6 = []
for i in range(len(appid)):
    if len(transport[transport.App_ID == appid[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index) >5:
        appid_6.append(appid[i])
        
        
for i in range(len(appid_6)):
    
    if i < len(appid_6)-1:
    # rule 2개만 뽑는다. 
        ss_1 = transport[transport.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[0]
        ss_2 = transport[transport.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[1]
       
        xx_1 = ((transport[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss_1)).sum(axis=1)==5)
        xx_2 = ((transport[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss_2)).sum(axis=1)==5)
        
    # rule 2개의 new_label을 다음 것으로 바꿔준다.     
        transport.loc[xx_1.index[xx_1], 'new_label'] = appid_6[i+1]
        transport.loc[xx_2.index[xx_2], 'new_label'] = appid_6[i+1]
        
    # 마지막 것 첫번재로 넘겨주는 것    
    else:
        ss_1 = transport[transport.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[0]
        ss_2 = transport[transport.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[1]
       
        xx_1 = ((transport[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss_1)).sum(axis=1)==5)
        xx_2 = ((transport[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss_2)).sum(axis=1)==5)
        
    # rule 2개의 new_label을 다음 것으로 바꿔준다.     
        transport.loc[xx_1.index[xx_1], 'new_label'] = appid_6[0]
        transport.loc[xx_2.index[xx_2], 'new_label'] = appid_6[0]

In [40]:
# -*- coding: utf-8 -*-


#======================================================================================================================#

# ----------------------- 0. 정확도 판단을 위해 appid rule별 데이터 만들기

# ----------------------- 2. IP encoding

transport_ip65536_delH = parallelize_dataframe(transport, ipencoding_65536_delH)

# ----------------------- 3. DNS encoding

transport_1 = data_nodns_unknown_del(transport)
transport_vector, transport_modelinit, max_length_transport = make_fasttext_model_vector2(transport_1)
transportdns_training = making_training_bydns(transport, transport_vector)

# ----------------------- 4.  ETC Features encoding

transport_age = age_onehot(transport)
transport_gender = gender_onehot(transport)
transport_port = port_onehot(transport)
transport_protocol = protocol_onehot(transport)
transport_lastdnstimetolive = train_lastdnstimetolive(transport)
transport_delta_dn_link_data_size = train_delta_dn_link_data_size(transport)
transport_delta_up_link_data_size = train_delta_up_link_data_size(transport)
transport_synackmsssize = train_synackmsssize(transport)
transport_delta_dn_link_payload_size = train_delta_dn_link_payload_size(transport)
transport_delta_up_link_payload_size = train_delta_up_link_payload_size(transport)

transport_training_ip65536_delH = pd.concat(
    [transport_ip65536_delH, transportdns_training, transport_port, transport_protocol,
                                             transport_lastdnstimetolive,transport_delta_dn_link_data_size,transport_delta_up_link_data_size,transport_synackmsssize,transport_delta_dn_link_payload_size,
                                             transport_delta_up_link_payload_size], axis=1)


In [41]:
transport_training_ip65536_delH = pd.concat([transport_training_ip65536_delH, transport[['App_ID']], transport[['new_label']]], axis = 1)

In [42]:
a = pd.DataFrame(columns = transport_training_ip65536_delH.columns)

for i in range(len(appid)):
    if len(transport_training_ip65536_delH[transport_training_ip65536_delH.new_label == appid[i]])<6:
        xx = transport_training_ip65536_delH.loc[list(transport_training_ip65536_delH[transport_training_ip65536_delH.new_label == appid[i]].index)]
        xx['anomaly_score'] = [2 for i in range(len(xx))]
        a = pd.concat([a,xx])

            
    else:
        clf = IsolationForest(n_estimators=50, contamination=0.2, random_state=35, verbose=1)
        clf.fit(transport_training_ip65536_delH[transport_training_ip65536_delH.new_label == appid[i]].drop(['App_ID','new_label'], axis = 1))
        y_pred = clf.score_samples(transport_training_ip65536_delH[transport_training_ip65536_delH.new_label == appid[i]].drop(['App_ID','new_label'], axis = 1)) #1,-1로 나온다.
        xx = transport_training_ip65536_delH.loc[list(transport_training_ip65536_delH[transport_training_ip65536_delH.new_label == appid[i]].index)]
        xx['anomaly_score'] = list(y_pred)
        a = pd.concat([a,xx])
        
        
###### -1이 이상치, 1이 정상.

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

In [43]:
transport = pd.concat([transport, a[['anomaly_score']]], axis = 1)

In [59]:
every = 0
anomaly = 0


for i in range(len(appid_6)):
    ss_1 = transport[transport.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[0]
    ss_2 = transport[transport.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[1]
       
    xx_1 = ((transport[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss_1)).sum(axis=1)==5)
    xx_2 = ((transport[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss_2)).sum(axis=1)==5)

        
    every += 2
        
        
    if sum(transport.loc[xx_1.index[xx_1], 'anomaly_score'] <-0.45)/len(transport.loc[xx_1.index[xx_1], 'anomaly_score']) >= 0.7:
        anomaly += 1
            
    if sum(transport.loc[xx_2.index[xx_2], 'anomaly_score'] <-0.45)/len(transport.loc[xx_2.index[xx_2], 'anomaly_score']) >= 0.7:
        anomaly += 1
      
    
    

In [49]:
algorithm = 0

for i in range(len(appid_6)):
    
    for j in range(len(transport[transport.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index)):
  
        ss = transport[transport.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[j]
        xx = ((transport[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss)).sum(axis=1)==5)
        
        
        if sum(xx) >= 6:
        # 이게 각 rule 별 anomaly_score다.    
            if sum(transport.loc[xx.index[xx], 'anomaly_score'] <-0.5)/len(transport.loc[xx.index[xx]]) >= 0.7:
                algorithm += 1

In [50]:
print(algorithm)

2658


In [51]:
algorithm = 0

for i in range(len(appid_6)):
    
    for j in range(len(transport[transport.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index)):
  
        ss = transport[transport.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[j]
        xx = ((transport[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss)).sum(axis=1)==5)
        
        
        if sum(xx) >= 6:
        # 이게 각 rule 별 anomaly_score다.    
            if sum(transport.loc[xx.index[xx], 'anomaly_score'] <-0.45)/len(transport.loc[xx.index[xx]]) >= 0.7:
                algorithm += 1

In [52]:
print(algorithm)

4139


In [53]:
algorithm = 0

for i in range(len(appid_6)):
    
    for j in range(len(transport[transport.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index)):
  
        ss = transport[transport.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[j]
        xx = ((transport[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss)).sum(axis=1)==5)
        
        
        if sum(xx) >= 6:
        # 이게 각 rule 별 anomaly_score다.    
            if sum(transport.loc[xx.index[xx], 'anomaly_score'] <-0.5)/len(transport.loc[xx.index[xx]]) >= 0.6:
                algorithm += 1

In [54]:
print(algorithm)

2778


In [55]:
algorithm = 0

for i in range(len(appid_6)):
    
    for j in range(len(transport[transport.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index)):
  
        ss = transport[transport.App_ID == appid_6[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index[j]
        xx = ((transport[['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']] ==list(ss)).sum(axis=1)==5)
        
        
        if sum(xx) >= 6:
        # 이게 각 rule 별 anomaly_score다.    
            if sum(transport.loc[xx.index[xx], 'anomaly_score'] <-0.45)/len(transport.loc[xx.index[xx]]) >= 0.6:
                algorithm += 1

KeyboardInterrupt: 

In [56]:
print(algorithm)

4147


In [47]:
everything = 0

for i in range(len(appid)):
    
    for j in range(len(transport[transport.App_ID == appid[i]].groupby(['Protocol', 'App_Port','DNS.QueryName', 'App_IPv4', 'App_IPv6']).count().index)):
  
        everything +=1

In [60]:
every, anomaly

(616, 566)

In [48]:
everything

35587